In [4]:
import requests

In [5]:
from bs4 import BeautifulSoup

### Retrieve and save HTML

In [6]:
url = "https://www.berlin.de/rbmskzl/"

In [7]:
response = requests.get(url)

In [8]:
response.status_code

200

In [9]:
html_text = response.text

In [10]:
html_text[:100]

'<!doctype html>\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="de" lang="de">\n<head prefix="og'

In [11]:
with open('berlin_rbmskzl_main_page.html', 'w') as openfile:
    openfile.write(html_text)

### Parse HTML (strukturierte parsen, one page)

In [12]:
with open('berlin_rbmskzl_main_page.html') as openfile:
    html_text = openfile.read()

In [13]:
soup = BeautifulSoup(html_text)

#### 1. Parse the top news section of the website

![berlin_de_top](berlin_de_top.png)

If we inspect the unredlying HTML code, we'll see that it is in a `<div>` element with a CSS class `'herounit-homepage herounit-homepage--default'`. Let's retrieve this div using BeautifulSoup syntax:

In [14]:
topdiv = soup.find("div", {"class": "herounit-homepage herounit-homepage--default"})

In [15]:
type(topdiv)

bs4.element.Tag

In [16]:
print(topdiv.prettify())

<div class="herounit-homepage herounit-homepage--default">
 <h1 class="title">
  Der Regierende Bürgermeister von Berlin - Senatskanzlei
 </h1>
 <div class="modul-buehne buehne--tileslayout">
  <ul class="buhne__list--teaser">
   <li>
    <div class="modul-teaser_buehne" data-add-clickable-area="smart">
     <div class="teaser_buehne__left">
      <div class="image">
       <!-- Image.view -->
       <div class="image__image image__image" style="">
        <img alt="Kai Wegner begrüßt Italiens Präsidenten Sergio Mattarella am 27.09.2024" class="jpg" data-orig="/rbmskzl/aktuelles/media/crop_nan_653_nan_340.48333740234375_1500_1000_54ccaad2ef03c22d8972e5b4746e29dc_p1_06454.jpg" loading="lazy" src="/imgscaler/yWuwttGbcIvWYHeN-MVnduJIEruYklTI1VYn1G_jO0w/rbig2zu1/L3N5czExLXByb2QvcmJtc2t6bC9ha3R1ZWxsZXMvbWVkaWEvY3JvcF9uYW5fNjUzX25hbl8zNDAuNDgzMzM3NDAyMzQzNzVfMTUwMF8xMDAwXzU0Y2NhYWQyZWYwM2MyMmQ4OTcyZTViNDc0NmUyOWRjX3AxXzA2NDU0LmpwZw.jpg?ts=1727715705"/>
       </div>
       <p class="image__c

In [17]:
topdiv_h2titles = topdiv.find_all('h2')

In [18]:
topdiv_h2titles

[<h2 class="title"> Kai Wegner begrüßt Italiens Präsidenten Sergio Mattarella</h2>,
 <h2 class="title">Wegner empfängt Jerusalemer Bürgermeister</h2>,
 <h2 class="title">Kai Wegner vor Ort</h2>,
 <h2 class="title">Fokusthemen</h2>]

In [19]:
topdiv_texts =  topdiv.find_all('p', {"class":"text"})

In [20]:
topdiv_texts

[<p class="text">
                     Der Regierende Bürgermeister begrüßte den italienischen Präsidenten und seine Frau in Berlin am Brandenburger Tor.                        </p>,
 <p class="text">
                 Kai Wegner, empfing am 26. September den Bürgermeister von Jerusalem, Mosche Lion.                        </p>,
 <p class="text">
                 Der Regierende Bürgermeister besucht am 28. Oktober den Bezirk Kreuzberg-Friedrichshain und stellt sich den Fragen der Berlinerinnen und Berliner. Melden Sie sich an!                        </p>,
 <p class="text">
                 Berlin steht vor diversen gesellschaftlichen Herausforderungen. Der Berliner Senat setzt einen Fokus auf sechs Themenschwerpunkte.                        </p>]

In [21]:
topdiv_texts = [x.text.strip() for x in topdiv_texts]

In [22]:
topdiv_texts

['Der Regierende Bürgermeister begrüßte den italienischen Präsidenten und seine Frau in Berlin am Brandenburger Tor.',
 'Kai Wegner, empfing am 26. September den Bürgermeister von Jerusalem, Mosche Lion.',
 'Der Regierende Bürgermeister besucht am 28. Oktober den Bezirk Kreuzberg-Friedrichshain und stellt sich den Fragen der Berlinerinnen und Berliner. Melden Sie sich an!',
 'Berlin steht vor diversen gesellschaftlichen Herausforderungen. Der Berliner Senat setzt einen Fokus auf sechs Themenschwerpunkte.']

In [23]:
topdiv_h2titles = [x.text.strip() for x in topdiv_h2titles]

In [24]:
resulting_data = []

for i, text in enumerate(topdiv_texts):
    new_entry = {}
    new_entry['title'] = topdiv_h2titles[i]
    new_entry['text'] = text
    resulting_data.append(new_entry)

In [25]:
resulting_data

[{'title': 'Kai Wegner begrüßt Italiens Präsidenten Sergio Mattarella',
  'text': 'Der Regierende Bürgermeister begrüßte den italienischen Präsidenten und seine Frau in Berlin am Brandenburger Tor.'},
 {'title': 'Wegner empfängt Jerusalemer Bürgermeister',
  'text': 'Kai Wegner, empfing am 26. September den Bürgermeister von Jerusalem, Mosche Lion.'},
 {'title': 'Kai Wegner vor Ort',
  'text': 'Der Regierende Bürgermeister besucht am 28. Oktober den Bezirk Kreuzberg-Friedrichshain und stellt sich den Fragen der Berlinerinnen und Berliner. Melden Sie sich an!'},
 {'title': 'Fokusthemen',
  'text': 'Berlin steht vor diversen gesellschaftlichen Herausforderungen. Der Berliner Senat setzt einen Fokus auf sechs Themenschwerpunkte.'}]

### Parse HTML for links

In [71]:
links = soup.find_all('a', href=True)

In [72]:
links[1]

<a href="/rbmskzl" title='Startseite von "Der Regierende Bürgermeister Senatskanzlei"'><span class="institution">Der Regierende Bürgermeister</span><span class="title">Senatskanzlei</span></a>

Separate internal and external links

In [73]:
internal_links = []
external_links = []

for link in links:
    link_url = link['href']
    if link_url.startswith('/'):
        internal_links.append(link_url)
    else:
        external_links.append(link_url)                           

In [74]:
external_links[:5]

['https://www.berlin.de',
 'https://twitter.com/RegBerlin',
 'https://www.facebook.com/RegBerlin/',
 'https://www.instagram.com/regberlin/',
 'https://www.linkedin.com/company/regberlin/']

In [75]:
internal_links[:5]

['/rbmskzl',
 '/rbmskzl/leichte-sprache/',
 '/rbmskzl/gebaerdensprache/',
 '/rbmskzl/barrierefreiheitserklaerung.879589.php',
 '/rbmskzl/barrierefreiheitserklaerung.879589.php#contactbfe']

*To be continued*